[View in Colaboratory](https://colab.research.google.com/github/sriram-ramanathan/ML_L1/blob/master/Semantic_Similarity_with_TF_Hub_Universal_Encoder.ipynb)

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Universal Sentence Encoder

Note: You can run this notebook [live in Colab](https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb) with zero setup.

This notebook illustrates how to access the Universal Sentence Encoder and use it for sentence similarity and sentence classification tasks.

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.


# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [2]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub

More detailed information about installing Tensorflow can be found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/).

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")

# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

# Semantic Textual Similarity Task Example

The embeddings produced by the Universal Sentence Encoder are approximately normalized. The semantic similarity of two sentences can be trivially computed as the inner product of the encodings.

In [5]:
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")


def run_and_plot(session_, input_tensor_, messages_, encoding_tensor):
  message_embeddings_ = session_.run(
      encoding_tensor, feed_dict={input_tensor_: messages})
  plot_similarity(messages_, message_embeddings_, 90)
  return message_embeddings_

## Similarity Visualized
Here we show the similarity in a heat map. The final graph is a 9x9 matrix where each entry `[i, j]` is colored based on the inner product of the encodings for sentence `i` and `j`.

In [0]:
'''messages = [
    "phones",
    "I like my phone",
    "My phone is not good.",
    "Your cellphone looks great.",
    

    "Weather",
    "Will it snow tomorrow?",
    "Recently a lot of hurricanes have hit the US",
    "Global warming is real",

    "Food",
    "An apple a day, keeps the doctors away",
    "Eating strawberries is healthy",
    "Is paleo better than keto?",

    "age",
    "How old are you?",
    "what is your age?",
]

messages=["Ten student teams with inventions to improve people's lives worldwide - from portable ventilators for resource-strapped hospitals, to wheelchair cushions that prevent sores, to multipurpose sleeping bags for refugees - on Saturday split awards totaling $92,500 at the annual MIT IDEAS Global Challenge showcase and awards ceremony",
 ' Grand prize winner of $15,000 went to Umbulizer, a team developing a low-cost, portable ventilator - made by modifying and simplifying the machinery - for patients in rural areas where medical resources are scarce and unreliable',
 ' "High-end ventilators are not affordable for hospitals in Pakistan',
 'We\'ve basically simplified the model" to make them less expensive, team member Moiz Imam, a senior in mechanical engineering, told MIT News',
 ' An additional nine winners, chosen by a team of judges that include seasoned entrepreneurs and venture capitalists, received four $10,000 awards and five $7,500 awards',
 ' Traditional ventilators come with about 20 different functions, which make them complex and expensive',
 'Second-hand ventilators, which low-resource hospitals tend to buy, can run up to $15,000 to $20,000, the Umbulizer team said',
 'The alternative is a $40 hand-operated ventilator, called an "artificial manual breathing unit," or AMBU bag, which requires constant manual pumping',
 ' "People end up getting those AMBU bags, which are dangerous for lungs because you cannot regulate the volume or the pressure," team member Wasay Anwer, a junior in electrical engineering and computer science at MIT, told MIT News',
 '"That\'s where our device comes in."  The Umbulizer team - including Sanchay Gupta, a Harvard Medical School student, and Hamza Khan, a Harvard Business School student - is developing an automated ventilator that keeps only four critical functions, reducing the price to around $2,000',
 '"We talked to doctors in Pakistan and they said 90 percent of the patients only need four of the core functions of traditional second-hand ventilators',
 'Our device provides those core functions at about 10 percent of the price," Gupta said',
 ' The team has a working prototype and aims to pilot the device at a major Pakistan hospital in the summer',
 '"The [IDEAS] prize money will go a long way to helping us developing the final version of that prototype," Imam said',
 '"It will also help us find local manufacturers, because our end goal is to mass manufacture this device."']
'''

messages=['Using a machine-learning system known as a deep neural network, MIT researchers have created thefirst model that can replicate human performance on auditory tasks such as identifying a musical genre',
 'This model, which consists of many layers of information-processing units that can be trained on huge volumes of data to perform specific tasks, was used by the researchers to shed light on how the human brain may be performing the same tasks',
 'What these models give us, for the first time, is machine systems that can perform sensory tasks that matter to humans and that do so at human levels, says Josh McDermott, the Frederick A',
 'Middleton Assistant Professor of Neuroscience in the Department of Brain and Cognitive Sciences at MIT and the senior author of the study',
 'Historically, this type of sensory processing has been difficult to understand, in part because we haven\xe2\x80\x99t really had a very clear theoretical foundation and a good way to develop models of what might be going on',
  'MIT graduate student Alexander Kell and Stanford University Assistant Professor Daniel Yamins are the paper’s lead authors.']
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)
with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  embeds=run_and_plot(session, similarity_input_placeholder, messages,
               similarity_message_encodings)
  print(embeds)

In [9]:
from sklearn.cluster import KMeans
import pandas as pd
def central_sentence(tvec_weights,n_cluster=2):
    X=pd.DataFrame(tvec_weights)
    est = KMeans(n_clusters=n_cluster)
    Y = pd.DataFrame(est.fit_predict(X), columns=['cluster ID'])
    Z = pd.DataFrame(est.cluster_centers_[Y['cluster ID']])

    result = pd.concat([X, Y, Z], axis=1)  
    Z.columns=[str(i)+'_Centroid' for i in  Z.columns]

    #calculation distance to centroid for each sentence 
    from sklearn.metrics.pairwise import paired_distances
    distances=paired_distances(X,Z)
    result['distances']=distances
    min_distance_id=list(result.groupby('cluster ID')['distances'].idxmin())
    
    return min_distance_id,result

In [71]:
pd.set_option('display.max_colwidth', -1)
min_dis,result=central_sentence(embeds,n_cluster=3)
result['messages']=messages
noun_cluster=find_noun_cluster(messages,result)

min_dis.sort()
summary=''
for i in min_dis:
  print(i,result['cluster ID'][i],messages[i])
  
  if result['cluster ID'][i]!=noun_cluster:
    if summary=='':
      summary=messages[i]
    else:
      summary=summary + '. ' + messages[i]
      
print(summary)
  

Using a machine-learning system known as a deep neural network, MIT researchers have created thefirst model that can replicate human performance on auditory tasks such as identifying a musical genre
0.03333333333333333
This model, which consists of many layers of information-processing units that can be trained on huge volumes of data to perform specific tasks, was used by the researchers to shed light on how the human brain may be performing the same tasks
0.0
What these models give us, for the first time, is machine systems that can perform sensory tasks that matter to humans and that do so at human levels, says Josh McDermott, the Frederick A
0.11764705882352941
Middleton Assistant Professor of Neuroscience in the Department of Brain and Cognitive Sciences at MIT and the senior author of the study
0.36363636363636365
Historically, this type of sensory processing has been difficult to understand, in part because we havenât really had a very clear theoretical foundation and a good w

Using a machine-learning system known as a deep neural network, MIT researchers have created thefirst model that can replicate human performance on auditory tasks such as identifying a musical genre
0.03333333333333333
This model, which consists of many layers of information-processing units that can be trained on huge volumes of data to perform specific tasks, was used by the researchers to shed light on how the human brain may be performing the same tasks
0.0
What these models give us, for the first time, is machine systems that can perform sensory tasks that matter to humans and that do so at human levels, says Josh McDermott, the Frederick A
0.11764705882352941
Middleton Assistant Professor of Neuroscience in the Department of Brain and Cognitive Sciences at MIT and the senior author of the study
0.36363636363636365
Historically, this type of sensory processing has been difficult to understand, in part because we havenât really had a very clear theoretical foundation and a good w

In [68]:
noun_cluster

0

In [43]:
import nltk
nltk.download("averaged_perceptron_tagger")
from nltk.tag import pos_tag

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /content/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [62]:
#finding noun cluster 
def find_noun_cluster(messages,result):
  noun_perc=[]
  for i in range(len(messages)):
    sentence = messages[i]
    tagged_sent = pos_tag(sentence.split())
    

    propernouns = [word for word,pos in tagged_sent if pos == 'NNP']
    print(sentence)
    nperc=(len(propernouns)/len(sentence.split()))
    noun_perc.append(nperc)
    print(nperc)

  result['noun_perc']=noun_perc
  cluster_noun_perc=result.groupby('cluster ID')['noun_perc'].agg('mean')
  return cluster_noun_perc.idxmax()

Using a machine-learning system known as a deep neural network, MIT researchers have created thefirst model that can replicate human performance on auditory tasks such as identifying a musical genre
0.03333333333333333
This model, which consists of many layers of information-processing units that can be trained on huge volumes of data to perform specific tasks, was used by the researchers to shed light on how the human brain may be performing the same tasks
0.0
What these models give us, for the first time, is machine systems that can perform sensory tasks that matter to humans and that do so at human levels, says Josh McDermott, the Frederick A
0.11764705882352941
Middleton Assistant Professor of Neuroscience in the Department of Brain and Cognitive Sciences at MIT and the senior author of the study
0.36363636363636365
Historically, this type of sensory processing has been difficult to understand, in part because we havenât really had a very clear theoretical foundation and a good w

0

In [46]:
from sklearn import metrics
def find_silhouette_score(tvec_weights):
    ss=[]
    nc=[]
    for n in range(2,int(len(tvec_weights)/4)):
        sentences_id,result=central_sentence(tvec_weights,n)
        ss.append(metrics.silhouette_score(tvec_weights,result['cluster ID'], metric='euclidean'))
        nc.append(n)
    zipped=find_silhouette_score(tvec_weights)
    zipped.sort(key = lambda t: t[1],reverse=True)
    optimal_k=zipped[0][0]
    
    return zip(nc,ss),optima
  
zipped,opti=find_silhouette_score(embeds)

## Evaluation: STS (Semantic Textual Similarity) Benchmark

The [**STS Benchmark**](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) provides an intristic evaluation of the degree to which similarity scores computed using sentence embeddings align with human judgements. The benchmark requires systems to return similarity scores for a diverse selection of sentence pairs. [Pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) is then used to evaluate the quality of the machine similarity scores against human judgements.

### Download data

In [0]:
import pandas
import scipy
import math


def load_sts_dataset(filename):
  # Loads a subset of the STS dataset into a DataFrame. In particular both
  # sentences and their human rated similarity score.
  sent_pairs = []
  with tf.gfile.GFile(filename, "r") as f:
    for line in f:
      ts = line.strip().split("\t")
      # (sent_1, sent_2, similarity_score)
      sent_pairs.append((ts[5], ts[6], float(ts[4])))
  return pandas.DataFrame(sent_pairs, columns=["sent_1", "sent_2", "sim"])


def download_and_load_sts_data():
  sts_dataset = tf.keras.utils.get_file(
      fname="Stsbenchmark.tar.gz",
      origin="http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz",
      extract=True)

  sts_dev = load_sts_dataset(
      os.path.join(os.path.dirname(sts_dataset), "stsbenchmark", "sts-dev.csv"))
  sts_test = load_sts_dataset(
      os.path.join(
          os.path.dirname(sts_dataset), "stsbenchmark", "sts-test.csv"))

  return sts_dev, sts_test


sts_dev, sts_test = download_and_load_sts_data()

### Build Evaluation Graph

In [0]:
text_a = sts_dev['sent_1'].tolist()
text_b = sts_dev['sent_2'].tolist()
dev_scores = sts_dev['sim'].tolist()
sts_input1 = tf.placeholder(tf.string, shape=(None))
sts_input2 = tf.placeholder(tf.string, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1))
sts_encode2 = tf.nn.l2_normalize(embed(sts_input2))
sim_scores = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)

### Evaluate Sentence Embeddings

In [0]:
def run_sts_benchmark(session):
  """Returns the similarity scores"""
  emba, embb, scores = session.run(
      [sts_encode1, sts_encode2, sim_scores],
      feed_dict={
          sts_input1: text_a,
          sts_input2: text_b
      })
  return scores


with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  scores = run_sts_benchmark(session)

pearson_correlation = scipy.stats.pearsonr(scores, dev_scores)
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))